<a href="https://colab.research.google.com/github/sajaldebnath/topic_sentiment_analysis/blob/main/TopicModeling_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **IISc CCE AI/ML Capstone Project - Topic modeling and sentiment analysis based on user review data.**

### **Purpose:**
This project aims to take user reviews as inputs and figure out the following:

*   Figure out topics discussed
*   Model the topics
*   Topic clustering
*   Unsupervised sentiment analysis on each topic cluster providing Top N positive, negative, and neutral sentiment topics
*   Figuring out the Top N impactful topic for overall rating improvement

### **Approach:**

We are taking a phased approach to achieve the goal. The following phases will be carried out:

#### **Milestone-1**

*  Cleaning and preparing data
*  Topic modeling
*  Unsupervised topic clustering
*  End of Milestone-1 - Top N trending topic

#### **Milestone-2**

* Unsupervised sentiment analysis on each topic cluster
* End of Milestone-2 - Top N positive, negative, and neutral sentiment topics

#### **Milestone-3**

* End of Milestone-3 - Top N impactful topic for overall rating improvement

### **Milestone-1**



1.   Import all the necessary libraries

In this dataset from Kaggle https://www.kaggle.com/datasets/rhonarosecortez/new-york-airbnb-open-data/data we have 3 files:


*   calendar.csv
*   listings.csv
*   reviewes.csv

We will be working only with reviews.csv as we want to do a topic modelling on the reviews.



In [1]:
# Importing all necessary libraries

# utilities
import re
import pickle
import numpy as np
import pandas as pd

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report


2. Import and view the dataset

In [6]:
# Importing the dataset
#COLUMNS  = ["listing_id", "id", "date", "reviewer_id", "reviewer_name", "comments"]
# ENCODING = "ISO-8859-1"
url = 'https://raw.githubusercontent.com/sajaldebnath/topic_sentiment_analysis/refs/heads/main/data/NY-Airbnb-Open-Data-Reviews.csv'
df = pd.read_csv(url, engine='python', on_bad_lines='skip' )

# df = pd.read_csv(url )

df.head(10) # view the first 10 rows of the dataframe

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2992450,15066586,2014-07-01,16827297,Kristen,Large apartment; nice kitchen and bathroom. Ke...
1,2992450,21810844,2014-10-24,22648856,Christopher,"This may be a little late, but just to say Ken..."
2,2992450,27434334,2015-03-04,45406,Altay,The apartment was very clean and convenient to...
3,2992450,28524578,2015-03-25,5485362,John,Kenneth was ready when I got there and arrange...
4,2992450,35913434,2015-06-23,15772025,Jennifer,We were pleased to see how 2nd Street and the ...
5,2992450,38893053,2015-07-19,11614467,Stephanie,"The flat is not in a good area, while we were ..."
6,2992450,57989144,2015-12-31,28580637,Betty,The apartment was centrally located near all t...
7,2992450,457366954464901293,2021-09-22,413779309,Carolina,The place is clean and the host is very nice
8,2992450,695544085190177036,2022-08-17,19928494,Kyle,It was much dirtier in person and half the fur...
9,3820211,17665203,2014-08-15,11024290,Abigail,We had a marvelous time staying at Terra's bea...


In [7]:
# Removing the unnecessary columns since we will only be working with reviews
# later on we want to co-relate reviews with listings and perhaps with the reviewers
data = df[['listing_id','reviewer_id', 'comments']]
# Replacing the values to ease understanding.
# dataset['sentiment'] = dataset['sentiment'].replace(4,1)

data.head(10)

,listing_id,reviewer_id,comments
0,2992450,16827297,Large apartment; nice kitchen and bathroom. Ke...
1,2992450,22648856,"This may be a little late, but just to say Ken..."
2,2992450,45406,The apartment was very clean and convenient to...
3,2992450,5485362,Kenneth was ready when I got there and arrange...
4,2992450,15772025,We were pleased to see how 2nd Street and the ...
5,2992450,11614467,"The flat is not in a good area, while we were ..."
6,2992450,28580637,The apartment was centrally located near all t...
7,2992450,413779309,The place is clean and the host is very nice
8,2992450,19928494,It was much dirtier in person and half the fur...
9,3820211,11024290,We had a marvelous time staying at Terra's bea...


In [8]:
# Next let's find out the details of the dataset
data.info() #provides a summary of the data frame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24752 entries, 0 to 24751
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   24752 non-null  int64 
 1   reviewer_id  24752 non-null  int64 
 2   comments     24745 non-null  object
dtypes: int64(2), object(1)
memory usage: 580.2+ KB


From the above information we can see that there are 24752 rows of data and without any null values. So, we don't have to worry about null data in this dataset.
Still let's handle that situation for any future datasets.

In [9]:
# Explicitly check for missing values
if (data.isnull().sum() > 0).any():
    print("Missing values found in the dataset.")
    # The best approach is to just drop them.
    data = data.dropna(subset=['comments'])
else:
    print("No missing values found in the dataset.")

data.head(10)
# data.info()

Missing values found in the dataset.


,listing_id,reviewer_id,comments
0,2992450,16827297,Large apartment; nice kitchen and bathroom. Ke...
1,2992450,22648856,"This may be a little late, but just to say Ken..."
2,2992450,45406,The apartment was very clean and convenient to...
3,2992450,5485362,Kenneth was ready when I got there and arrange...
4,2992450,15772025,We were pleased to see how 2nd Street and the ...
5,2992450,11614467,"The flat is not in a good area, while we were ..."
6,2992450,28580637,The apartment was centrally located near all t...
7,2992450,413779309,The place is clean and the host is very nice
8,2992450,19928494,It was much dirtier in person and half the fur...
9,3820211,11024290,We had a marvelous time staying at Terra's bea...


3. In this section we will handle the pre-processing of the date which includes the following:

* **Lower Casing**: Each text is converted to lowercase.

* **Replacing URLs**: Links starting with "http" or "https" or "www" are replaced by "URL".

* **Replacing Emojis**: Replace emojis by using a pre-defined dictionary containing emojis along with their meaning. (eg: ":)" to "EMOJIsmile")

* **Replacing Usernames**: Replace @Usernames with word "USER". (eg: "@Kaggle" to "USER")

* **Removing Non-Alphabets**: Replacing characters except Digits and Alphabets with a space.

* **Removing Consecutive letters**: 3 or more consecutive letters are replaced by 2 letters. (eg: "Heyyyy" to "Heyy")

* **Removing Short Words**: Words with length less than 2 are removed.

* **Removing Stopwords**: Stopwords are the English words which does not add much meaning to a sentence. They can safely be ignored without sacrificing the meaning of the sentence. (eg: "the", "he", "have")

* **Lemmatizing**: Lemmatization is the process of converting a word to its base form. (e.g: “Great” to “Good”)


In [10]:
# Defining dictionary containing all emojis with their meanings.
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad',
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed',
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink',
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

In [11]:
def preprocess_comments(comments):

    # Defining regex patterns.
    urlPattern        = r'http[s]?://\S+' # Removes any website link starting with http or https
    userPattern       = '@[^\s]+'         # Removes any users names starting with @ character
    alphaPattern      = "[^a-zA-Z0-9]"    # Removes text except alpha numeric patterns
    sequencePattern   = r"(.)\1\1+"       # Finds out 3 or more repeating letters likes "aaa"
    seqReplacePattern = r"\1\1"           # Replace the repeating letter sequence with two letters
    symbolPattern = r'Ã[^\x80-\xBF]+'     # Finds out the special characters like Ã
    spacePattern = r'\s+'                 # Finds out extra spaces betweem words


    # Replace all URls with ' '
    comments = re.sub(urlPattern, '', comments, flags=re.MULTILINE)
    # Replace @USERNAME to ' '.
    comments = re.sub(userPattern,' ', comments)
    # Replace all non alphabets.
    comments = re.sub(alphaPattern, " ", comments)
    # Replace 3 or more consecutive letters by 2 letters.
    comments = re.sub(sequencePattern, seqReplacePattern, comments)
    # Replace special encoded symbols that appear as characters like Ã, Å, Ë
    comments = re.sub(symbolPattern, ' ', comments)

    # Replace all emojis.
    for emoji in emojis.keys():
      comments = comments.replace(emoji, " " + emojis[emoji])

    # Replace multiple spaces with a single space
    comments = re.sub(spacePattern, ' ', comments)
    # Remove all leading or trailing spaces
    comments = comments.strip()

    return comments.lower()  # Convert all to lowercase

In [12]:
import time
t = time.time()
data['comments'] = data['comments'].apply(preprocess_comments) #applying the preprocess_comments function to comments column

print(f'Text Preprocessing complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

Text Preprocessing complete.
Time Taken: 2 seconds


In [13]:
#VIEW THE CLEANED Comments IN "comments" column again
pd.set_option('display.max_colwidth', None) #to view text fully and so that
#pandas doesn't truncate the text

data.head(10) # Printing the data to see the updated result

,listing_id,reviewer_id,comments
0,2992450,16827297,large apartment nice kitchen and bathroom kenneth left drinks for us which was so nice his cousin lives upstairs and she was very nice and helpful too the internet only worked about half the time and everything on the street can be heard from the main bedroom wonderful stay it was exactly what we needed it is not in the best area of town but i never felt unsafe
1,2992450,22648856,this may be a little late but just to say kenneth was quick to respond to our request he left some supplies in the refrigerator and kindly showed us round downtown albany we were without a car and he arranged for his cousin kristina to take us to supermarket br the apartment is large clean with many original features it is situated in an historical area that is fairly central although like the rest of albany you have to climb a steep hill to get to it if you are on foot br
2,2992450,45406,the apartment was very clean and convenient to downtown one thing to keep in mind one bedroom is huge and the other is tiny so if you re traveling with a friend you might have to flip a coin
3,2992450,5485362,kenneth was ready when i got there and arranged for the upstairs neighbor to meet me at the door with the keys shortly after that i was left on my own in the privacy of this large 2 br apartment just like i like it i really like my privacy and there was no time at which anyone came to visit me or bother me unannounced at one point kenneth had to pic up a refrigerator stored in the apartment but the neighbor lady gave me plenty of notice and i was on my way out before he got there when i returned the extra refrigerator was gone and i had the nice quiet apartment to myself at one point i went to take a shower and found some body wash in a drawer that was nice since all i brought with me was shampoo the apartment is a city street apartment meaning that like many two family homes the br window is feet away from the public sidewalk and you can hear people all hours of the night walking by and conversing but its not terribly disruptive overall the apartment was just what i needed and i had an enjoyable time while there the heat worked perfectly and the apartment was picked up the bed was a king size and very comfortable with sensible pillows and coverings the water pressure on the shower was awesome
4,2992450,15772025,we were pleased to see how 2nd street and the ten broeck neighborhood in albany have come along its great to see all of the gorgeous brownstones including the one we stayed in being renovated and cared for br in terms of quality i was pleased by the courtesy of the host s friend who lived nearby he was there to let us in and make sure i knew to call with any problems with a full kitchen and two bedrooms my daughter and i felt at home with plenty of space as a native of albany i m used to thin windows and occasionally noisy neighbors if you don t sleep well don t take the master bedroom and be aware that loud music may be playing during the evening and weekend but there is a fan ac unit to drown out the din from the street br this space has been renovated and a vast improvement i m sure from where it was and i m hoping progress will continue here as it seemed a few projects were incomplete i would have loved to enjoy more sunlight in the house but there was a ton of tree debris in the back yard so there was no way to enjoy opening the windows br on my own this place was fine but if traveling with my child again i will probably pick a quieter location where i d be able to open windows or put her down for a nap without worrying about excessive noise
5,2992450,11614467,the flat is not in a good area while we were fine you do have to be careful even our cab driver told us that i wouldn t stay again the flat also does not have air con the large bedroom had an obnoxiously loud fan which kept the one room cool the rest of the house is very hot so if you are going to stay here bear these things in mind
6,2992450

In [15]:
# Tokenize using nltk library
# pip install nltk # If not already present then install the module
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

def tokenize_comment(comment):
    tokens = word_tokenize(comment) # Break up and tokenize the comment
    return tokens

# Calling the tokenize_comment function on the comment to tokenize it
# Also adding another column called "Tokens" to store the tokenized words
data['Tokens'] = data['comments'].apply(tokenize_comment)

print(f'Tokenization complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Tokenization complete.
Time Taken: 98 seconds


In [ ]:
# Let's check the tokenized data
data.head(10)

,listing_id,reviewer_id,comments,Tokens
0,2992450,16827297,large apartment nice kitchen and bathroom kenneth left drinks for us which was so nice his cousin lives upstairs and she was very nice and helpful too the internet only worked about half the time and everything on the street can be heard from the main bedroom wonderful stay it was exactly what we needed it is not in the best area of town but i never felt unsafe,"[large, apartment, nice, kitchen, and, bathroom, kenneth, left, drinks, for, us, which, was, so, nice, his, cousin, lives, upstairs, and, she, was, very, nice, and, helpful, too, the, internet, only, worked, about, half, the, time, and, everything, on, the, street, can, be, heard, from, the, main, bedroom, wonderful, stay, it, was, exactly, what, we, needed, it, is, not, in, the, best, area, of, town, but, i, never, felt, unsafe]"
1,2992450,22648856,this may be a little late but just to say kenneth was quick to respond to our request he left some supplies in the refrigerator and kindly showed us round downtown albany we were without a car and he arranged for his cousin kristina to take us to supermarket br the apartment is large clean with many original features it is situated in an historical area that is fairly central although like the rest of albany you have to climb a steep hill to get to it if you are on foot br,"[this, may, be, a, little, late, but, just, to, say, kenneth, was, quick, to, respond, to, our, request, he, left, some, supplies, in, the, refrigerator, and, kindly, showed, us, round, downtown, albany, we, were, without, a, car, and, he, arranged, for, his, cousin, kristina, to, take, us, to, supermarket, br, the, apartment, is, large, clean, with, many, original, features, it, is, situated, in, an, historical, area, that, is, fairly, central, although, like, the, rest, of, albany, you, have, to, climb, a, steep, hill, to, get, to, it, if, you, are, on, foot, br]"
2,2992450,45406,the apartment was very clean and convenient to downtown one thing to keep in mind one bedroom is huge and the other is tiny so if you re traveling with a friend you might have to flip a coin,"[the, apartment, was, very, clean, and, convenient, to, downtown, one, thing, to, keep, in, mind, one, bedroom, is, huge, and, the, other, is, tiny, so, if, you, re, traveling, with, a, friend, you, might, have, to, flip, a, coin]"
3,2992450,5485362,kenneth was ready when i got there and arranged for the upstairs neighbor to meet me at the door with the keys shortly after that i was left on my own in the privacy of this large 2 br apartment just like i like it i really like my privacy and there was no time at which anyone came to visit me or bother me unannounced at one point kenneth had to pic up a refrigerator stored in the apartment but the neighbor lady gave me plenty of notice and i was on my way out before he got there when i returned the extra refrigerator was gone and i had the nice quiet apartment to myself at one point i went to take a shower and found some body wash in a drawer that was nice since all i brought with me was shampoo the apartment is a city street apartment meaning that like many two family homes the br window is feet away from the public sidewalk and you can hear people all hours of the night walking by and conversing but its not terribly disruptive overall the apartment was just what i needed and i had an enjoyable time while there the heat worked perfectly and the apartment was picked up the bed was a king size and very comfortable with sensible pillows and coverings the water pressure on the shower was awesome,"[kenneth, was, ready, when, i, got, there, and, arranged, for, the, upstairs, neighbor, to, meet, me, at, the, door, with, the, keys, shortly, after, that, i, was, left, on, my, own, in, the, privacy, of, this, large, 2, br, apartment, just, like, i, like, it, i, really, like, my, privacy, and, there, was, no, time, at, which, anyone, came, to, visit, me, or, bother, me, unannounced, at, one, po

**Removing the Stopwords**:

In this section we will remove all the defined stopwords

In [16]:
# Download the stopwords list
nltk.download('stopwords')
from nltk.corpus import stopwords

# Define the function to remove the stopword from the tokens
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))  # Get the set of English stopwords

    if custom_stopwordlist:
        stop_words.update(custom_stopwordlist)
    return [word for word in tokens if word.lower() not in stop_words]  # Filter out stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
#Let's define a list of custom_stop_words which does not add value to the topic
custom_stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're',
             's', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

# Remove stopwords from the Tokens column
data['Filtered_Tokens'] = data['Tokens'].apply(remove_stopwords)


print(f'Removing stopwords complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

Removing stopwords complete.
Time Taken: 114 seconds


In [18]:
# Let's check the updated data again
data.head(10)

,listing_id,reviewer_id,comments,Tokens,Filtered_Tokens
0,2992450,16827297,large apartment nice kitchen and bathroom kenneth left drinks for us which was so nice his cousin lives upstairs and she was very nice and helpful too the internet only worked about half the time and everything on the street can be heard from the main bedroom wonderful stay it was exactly what we needed it is not in the best area of town but i never felt unsafe,"[large, apartment, nice, kitchen, and, bathroom, kenneth, left, drinks, for, us, which, was, so, nice, his, cousin, lives, upstairs, and, she, was, very, nice, and, helpful, too, the, internet, only, worked, about, half, the, time, and, everything, on, the, street, can, be, heard, from, the, main, bedroom, wonderful, stay, it, was, exactly, what, we, needed, it, is, not, in, the, best, area, of, town, but, i, never, felt, unsafe]","[large, apartment, nice, kitchen, bathroom, kenneth, left, drinks, us, nice, cousin, lives, upstairs, nice, helpful, internet, worked, half, time, everything, street, heard, main, bedroom, wonderful, stay, exactly, needed, best, area, town, never, felt, unsafe]"
1,2992450,22648856,this may be a little late but just to say kenneth was quick to respond to our request he left some supplies in the refrigerator and kindly showed us round downtown albany we were without a car and he arranged for his cousin kristina to take us to supermarket br the apartment is large clean with many original features it is situated in an historical area that is fairly central although like the rest of albany you have to climb a steep hill to get to it if you are on foot br,"[this, may, be, a, little, late, but, just, to, say, kenneth, was, quick, to, respond, to, our, request, he, left, some, supplies, in, the, refrigerator, and, kindly, showed, us, round, downtown, albany, we, were, without, a, car, and, he, arranged, for, his, cousin, kristina, to, take, us, to, supermarket, br, the, apartment, is, large, clean, with, many, original, features, it, is, situated, in, an, historical, area, that, is, fairly, central, although, like, the, rest, of, albany, you, have, to, climb, a, steep, hill, to, get, to, it, if, you, are, on, foot, br]","[may, little, late, say, kenneth, quick, respond, request, left, supplies, refrigerator, kindly, showed, us, round, downtown, albany, without, car, arranged, cousin, kristina, take, us, supermarket, br, apartment, large, clean, many, original, features, situated, historical, area, fairly, central, although, like, rest, albany, climb, steep, hill, get, foot, br]"
2,2992450,45406,the apartment was very clean and convenient to downtown one thing to keep in mind one bedroom is huge and the other is tiny so if you re traveling with a friend you might have to flip a coin,"[the, apartment, was, very, clean, and, convenient, to, downtown, one, thing, to, keep, in, mind, one, bedroom, is, huge, and, the, other, is, tiny, so, if, you, re, traveling, with, a, friend, you, might, have, to, flip, a, coin]","[apartment, clean, convenient, downtown, one, thing, keep, mind, one, bedroom, huge, tiny, traveling, friend, might, flip, coin]"
3,2992450,5485362,kenneth was ready when i got there and arranged for the upstairs neighbor to meet me at the door with the keys shortly after that i was left on my own in the privacy of this large 2 br apartment just like i like it i really like my privacy and there was no time at which anyone came to visit me or bother me unannounced at one point kenneth had to pic up a refrigerator stored in the apartment but the neighbor lady gave me plenty of notice and i was on my way out before he got there when i returned the extra refrigerator was gone and i had the nice quiet apartment to myself at one point i went to take a shower and found some body wash in a drawer that was nice since all i brought with me was shampoo the apartment is a city street apartment meaning that like many two family homes the br window is feet away from the public sidew

#### **4. STEMMING**

Stemming reduces words to their root form (or stem). Note: may not always be a valid word in the language.

**For example**:
* "playing" might be stemmed to "play"

* "studying" might be stemmed to "study"

* "running" might be stemmed to "run"

* "ring" might be stemmed to "r" (ineffiency)

#### **5. LEMMATIZATION**

Lemmatization reduces words to their base or dictionary form (lemma) by considering their dictionary meaning hence providing more accuracy than stemming.

For example:

* "running" becomes "run,"

* "better" becomes "good."

* "ring" remains "ring."

Hence I won't be performing Stemming, I'll be going forward with Lemmatization.

In [19]:
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to perform lemmatization
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(word, pos='v') for word in tokens]

#pos is set to v (verb) for better accuracy. Refer below manual to understand more.
# refer: https://www.nltk.org/book/ch05.html

data['Lemmatized_Tokens'] = data['Filtered_Tokens'].apply(lemmatize_tokens)

print(f'Lemmatization complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

[nltk_data] Downloading package wordnet to /root/nltk_data...


Lemmatization complete.
Time Taken: 133 seconds


In [20]:
# Let's see the updated result
data.head(10)

,listing_id,reviewer_id,comments,Tokens,Filtered_Tokens,Lemmatized_Tokens
0,2992450,16827297,large apartment nice kitchen and bathroom kenneth left drinks for us which was so nice his cousin lives upstairs and she was very nice and helpful too the internet only worked about half the time and everything on the street can be heard from the main bedroom wonderful stay it was exactly what we needed it is not in the best area of town but i never felt unsafe,"[large, apartment, nice, kitchen, and, bathroom, kenneth, left, drinks, for, us, which, was, so, nice, his, cousin, lives, upstairs, and, she, was, very, nice, and, helpful, too, the, internet, only, worked, about, half, the, time, and, everything, on, the, street, can, be, heard, from, the, main, bedroom, wonderful, stay, it, was, exactly, what, we, needed, it, is, not, in, the, best, area, of, town, but, i, never, felt, unsafe]","[large, apartment, nice, kitchen, bathroom, kenneth, left, drinks, us, nice, cousin, lives, upstairs, nice, helpful, internet, worked, half, time, everything, street, heard, main, bedroom, wonderful, stay, exactly, needed, best, area, town, never, felt, unsafe]","[large, apartment, nice, kitchen, bathroom, kenneth, leave, drink, us, nice, cousin, live, upstairs, nice, helpful, internet, work, half, time, everything, street, hear, main, bedroom, wonderful, stay, exactly, need, best, area, town, never, felt, unsafe]"
1,2992450,22648856,this may be a little late but just to say kenneth was quick to respond to our request he left some supplies in the refrigerator and kindly showed us round downtown albany we were without a car and he arranged for his cousin kristina to take us to supermarket br the apartment is large clean with many original features it is situated in an historical area that is fairly central although like the rest of albany you have to climb a steep hill to get to it if you are on foot br,"[this, may, be, a, little, late, but, just, to, say, kenneth, was, quick, to, respond, to, our, request, he, left, some, supplies, in, the, refrigerator, and, kindly, showed, us, round, downtown, albany, we, were, without, a, car, and, he, arranged, for, his, cousin, kristina, to, take, us, to, supermarket, br, the, apartment, is, large, clean, with, many, original, features, it, is, situated, in, an, historical, area, that, is, fairly, central, although, like, the, rest, of, albany, you, have, to, climb, a, steep, hill, to, get, to, it, if, you, are, on, foot, br]","[may, little, late, say, kenneth, quick, respond, request, left, supplies, refrigerator, kindly, showed, us, round, downtown, albany, without, car, arranged, cousin, kristina, take, us, supermarket, br, apartment, large, clean, many, original, features, situated, historical, area, fairly, central, although, like, rest, albany, climb, steep, hill, get, foot, br]","[may, little, late, say, kenneth, quick, respond, request, leave, supply, refrigerator, kindly, show, us, round, downtown, albany, without, car, arrange, cousin, kristina, take, us, supermarket, br, apartment, large, clean, many, original, feature, situate, historical, area, fairly, central, although, like, rest, albany, climb, steep, hill, get, foot, br]"
2,2992450,45406,the apartment was very clean and convenient to downtown one thing to keep in mind one bedroom is huge and the other is tiny so if you re traveling with a friend you might have to flip a coin,"[the, apartment, was, very, clean, and, convenient, to, downtown, one, thing, to, keep, in, mind, one, bedroom, is, huge, and, the, other, is, tiny, so, if, you, re, traveling, with, a, friend, you, might, have, to, flip, a, coin]","[apartment, clean, convenient, downtown, one, thing, keep, mind, one, bedroom, huge, tiny, traveling, friend, might, flip, coin]","[apartment, clean, convenient, downtown, one, thing, keep, mind, one, bedroom, huge, tiny, travel, friend, might, flip, coin]"
3,2992450,5485362,kenneth was ready when i got there and arranged for the upstairs neighb

### FEATURE EXTRACTION using TF-IDF

TF-IDF is a natural language processing (NLP) technique that's used to evaluate the importance of different words in a sentence.

It's simple, intuitive technique which utilizes a fixed size input. It captures the importance of words (hence the semantics) while reducing the influence of stopwords alongside.

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

#performing TF-IDF vectorization
def apply_tfidf(data):
    data['TFIDF_Tokens'] = data['Lemmatized_Tokens'].apply(lambda x: ' '.join(x)) # Creating a column TFIDF_Tokens from the Lammatized_Tokens column
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
    tfidf_matrix = tfidf_vectorizer.fit_transform(data['TFIDF_Tokens'])
    feature_names = tfidf_vectorizer.get_feature_names_out()
    return tfidf_matrix, feature_names

# Applying TF-IDF on our Dataframe
tfidf_matrix, feature_names = apply_tfidf(data)

print(f'Feature extraction using TF-IDF complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

Feature extraction using TF-IDF complete.
Time Taken: 140 seconds


In [22]:
# Printing the updated data
data.head(10)

,listing_id,reviewer_id,comments,Tokens,Filtered_Tokens,Lemmatized_Tokens,TFIDF_Tokens
0,2992450,16827297,large apartment nice kitchen and bathroom kenneth left drinks for us which was so nice his cousin lives upstairs and she was very nice and helpful too the internet only worked about half the time and everything on the street can be heard from the main bedroom wonderful stay it was exactly what we needed it is not in the best area of town but i never felt unsafe,"[large, apartment, nice, kitchen, and, bathroom, kenneth, left, drinks, for, us, which, was, so, nice, his, cousin, lives, upstairs, and, she, was, very, nice, and, helpful, too, the, internet, only, worked, about, half, the, time, and, everything, on, the, street, can, be, heard, from, the, main, bedroom, wonderful, stay, it, was, exactly, what, we, needed, it, is, not, in, the, best, area, of, town, but, i, never, felt, unsafe]","[large, apartment, nice, kitchen, bathroom, kenneth, left, drinks, us, nice, cousin, lives, upstairs, nice, helpful, internet, worked, half, time, everything, street, heard, main, bedroom, wonderful, stay, exactly, needed, best, area, town, never, felt, unsafe]","[large, apartment, nice, kitchen, bathroom, kenneth, leave, drink, us, nice, cousin, live, upstairs, nice, helpful, internet, work, half, time, everything, street, hear, main, bedroom, wonderful, stay, exactly, need, best, area, town, never, felt, unsafe]",large apartment nice kitchen bathroom kenneth leave drink us nice cousin live upstairs nice helpful internet work half time everything street hear main bedroom wonderful stay exactly need best area town never felt unsafe
1,2992450,22648856,this may be a little late but just to say kenneth was quick to respond to our request he left some supplies in the refrigerator and kindly showed us round downtown albany we were without a car and he arranged for his cousin kristina to take us to supermarket br the apartment is large clean with many original features it is situated in an historical area that is fairly central although like the rest of albany you have to climb a steep hill to get to it if you are on foot br,"[this, may, be, a, little, late, but, just, to, say, kenneth, was, quick, to, respond, to, our, request, he, left, some, supplies, in, the, refrigerator, and, kindly, showed, us, round, downtown, albany, we, were, without, a, car, and, he, arranged, for, his, cousin, kristina, to, take, us, to, supermarket, br, the, apartment, is, large, clean, with, many, original, features, it, is, situated, in, an, historical, area, that, is, fairly, central, although, like, the, rest, of, albany, you, have, to, climb, a, steep, hill, to, get, to, it, if, you, are, on, foot, br]","[may, little, late, say, kenneth, quick, respond, request, left, supplies, refrigerator, kindly, showed, us, round, downtown, albany, without, car, arranged, cousin, kristina, take, us, supermarket, br, apartment, large, clean, many, original, features, situated, historical, area, fairly, central, although, like, rest, albany, climb, steep, hill, get, foot, br]","[may, little, late, say, kenneth, quick, respond, request, leave, supply, refrigerator, kindly, show, us, round, downtown, albany, without, car, arrange, cousin, kristina, take, us, supermarket, br, apartment, large, clean, many, original, feature, situate, historical, area, fairly, central, although, like, rest, albany, climb, steep, hill, get, foot, br]",may little late say kenneth quick respond request leave supply refrigerator kindly show us round downtown albany without car arrange cousin kristina take us supermarket br apartment large clean many original feature situate historical area fairly central although like rest albany climb steep hill get foot br
2,2992450,45406,the apartment was very clean and convenient to downtown one thing to keep in mind one bedroom is huge and the other is tiny so if you re traveling with a friend you might have to flip a coin,"[the, apartment, was, very, clean, a

In [24]:
print(tfidf_matrix)

  (0, 4996)	0.17493895706601892
  (0, 835)	0.10336809175366515
  (0, 5933)	0.2952868536508733
  (0, 4939)	0.13636998165248565
  (0, 1301)	0.14864019467475828
  (0, 4911)	0.3385818901810859
  (0, 5038)	0.16873962397202252
  (0, 3166)	0.20351402233787116
  (0, 2726)	0.31036372323197925
  (0, 5109)	0.16101447498687257
  (0, 9272)	0.19728951203597714
  (0, 4412)	0.1327845581536683
  (0, 4745)	0.2352195420759967
  (0, 9846)	0.14278728113703198
  (0, 4331)	0.2365843162692512
  (0, 8723)	0.11719582114564309
  (0, 8168)	0.12763939570241142
  (0, 4389)	0.1955346554298571
  (0, 5235)	0.20009978769133852
  (0, 1350)	0.16693825343693347
  (0, 9829)	0.13304397656674702
  (0, 8077)	0.060325931473549727
  (0, 3426)	0.15093787940262393
  (0, 5854)	0.10277786929411963
  (0, 1423)	0.15618334414333815
  :	:
  (24743, 5854)	0.14612517483588747
  (24743, 5107)	0.19635253584260765
  (24743, 3139)	0.19344261515478092
  (24743, 5958)	0.18776252469287238
  (24743, 4257)	0.09269850925606873
  (24743, 4508)	0.12

In [25]:
print(feature_names)

['00' '00am' '00pm' ... 'zweiten' 'zwischenstop' 'zwischenstopp']
